In [ ]:
import pandas as pd

In [ ]:
summary = pd.read_csv("data/clinvar/variant_summary.txt",delimiter="\t")

In [ ]:
goodStatus = ["criteria provided, single submitter",
              "criteria provided, multiple submitters, no conflicts",
              "reviewed by expert panel",
              "practice guideline",
              ]

get all variants which have the review status we are interested in, are SNVs, and the name field is in the form I'm expecting.

In [ ]:
summary

In [ ]:
missense = summary[(summary.ReviewStatus.isin(goodStatus)) & \
                   (summary.Type == "single nucleotide variant") & \
                   (summary.Name.str.contains("\(p.")) & \
                   (~summary.Name.str.contains("=")) & \
                   (summary.Assembly.str.contains("38"))]

In [ ]:
missense

for each SNV, get the canonical sequence

In [ ]:
from processBioDBs.utilities import getSequence

In [ ]:
from easydict import EasyDict

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
sequences = EasyDict()

In [ ]:
for symbol in tqdm(missense.GeneSymbol.unique()):
    if symbol not in sequences:
        try:
            sequences[symbol] = getSequence(symbol)
        except:
            continue

Dump to pickle file

In [ ]:
import pickle

In [ ]:
with open("data/clinvar/seqs.pkl","wb") as f:
    pickle.dump(sequences,f)

reload pickle (if didn't just process)

In [ ]:
sequences = pickle.load(open("data/clinvar/seqs.pkl","rb"))

1 letter to 3 letter AA conversion table

In [ ]:
aatable = {"Ala":"A",
               "Arg": "R",
               "Asn": "N",
               "Asp": "D",
               "Cys": "C",
               "Gln": "Q",
               "Glu": "E",
               "Gly": "G",
               "His": "H",
               "Ile": "I",
               "Leu": "L",
               "Lys": "K",
               "Met": "M",
               "Phe": "F",
               "Pro": "P",
               "Ser": "S",
               "Thr": "T",
               "Trp": "W",
               "Tyr": "Y",
               "Val": "V"}

Get the variant from the name (expecting the form: \<original amino acid\>\<location\>\<new amino acid\>)

In [ ]:
missense = missense.assign(variant=missense.Name.apply(lambda s: s[s.find("(p.") + 3:-1]))

missense = missense.assign(og=missense.variant.str.slice(0,3))
missense = missense.assign(loc=missense.variant.str.slice(3,-3))
missense = missense.assign(newAA=missense.variant.str.slice(-3,))

In [ ]:
missense

drop rows where the original or new amino acid aren't valid

In [ ]:
badRows = set()

In [ ]:
for i,k in enumerate(missense.og):
    if k not in aatable:
         badRows.update([i])

In [ ]:
for i,k in enumerate(missense.newAA):
    if k not in aatable:
        badRows.update([i])

In [ ]:
missense = missense.drop(index=missense.index[list(badRows)])

make sure there are no duplicates

In [ ]:
missense = missense.drop_duplicates(subset=["Name"])

In [ ]:
missense

Find the rows where the canonical sequence I found match with the original amino acid given in the summary table

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import numpy as np

In [ ]:
matches = np.zeros(missense.shape[0])
for i,(index,r) in tqdm(enumerate(missense.iterrows())):
    if r.GeneSymbol in sequences:
        s = sequences[r.GeneSymbol]
        loc = int(r["loc"]) - 1
        if loc >= len(s):
            continue
        if s[loc] == aatable[r.og]:
            matches[i] = 1
        

In [ ]:
missense = missense.assign(match=matches)

In [ ]:
missense

In [ ]:
df = missense[missense.match == 1]

In [ ]:
df

In [ ]:
df = df.loc[(df['ClinicalSignificance'] == 'Uncertain significance')|
            (df['ClinicalSignificance'] == 'Likely benign')|
            (df['ClinicalSignificance'] == 'Benign')|
            (df['ClinicalSignificance'] == 'Pathogenic')|
            (df['ClinicalSignificance'] == 'Likely pathogenic')|
            (df['ClinicalSignificance'] == 'Benign/Likely benign')|
            (df['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')]

In [ ]:
df

In [ ]:
df = df.assign(label=(df.ClinicalSignificance.str.contains("pathogenic")) | \
               (df.ClinicalSignificance.str.contains("Pathogenic")))

In [ ]:
df